---
## 1. Setup Prostředí

In [ ]:
# Instalace (pro Colab)
!pip install pandas numpy scikit-learn joblib matplotlib seaborn tqdm -q

print("✓ Knihovny nainstalovány")

In [ ]:
# Import knihoven
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
import joblib
import json

# Scikit-learn
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve
)

warnings.filterwarnings('ignore')
np.random.seed(42)

# Nastavení grafů pro publikaci
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

print("✓ Knihovny načteny")

In [ ]:
# Připojení Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_PATH = '/content/drive/MyDrive/MachineLearning'
    RUNNING_ON_COLAB = True
    print(f"✓ Google Drive připojen: {DRIVE_PATH}")
except:
    DRIVE_PATH = '.'
    RUNNING_ON_COLAB = False
    print("ℹ️ Lokální prostředí")

# Cesty
DATA_PATH = f"{DRIVE_PATH}/data"
MODEL_PATH = f"{DRIVE_PATH}/models"
FIGURES_PATH = f"{DRIVE_PATH}/figures"

os.makedirs(FIGURES_PATH, exist_ok=True)
print(f"📁 Grafy budou uloženy do: {FIGURES_PATH}")

---
## 2. Načtení Dat a Modelů

In [ ]:
# Načtení dat
complete_path = f"{DATA_PATH}/complete/all_sectors_complete_10y.csv"
df = pd.read_csv(complete_path, parse_dates=['date'])

print(f"📈 Dataset: {len(df):,} záznamů")
print(f"📅 Období: {df['date'].min().strftime('%Y-%m')} až {df['date'].max().strftime('%Y-%m')}")
print(f"🏢 Sektory: {df['sector'].nunique()}")
print(f"📊 Tickery: {df['ticker'].nunique()}")

In [ ]:
# Načtení modelů
try:
    classifier = joblib.load(f"{MODEL_PATH}/price_classifier_tuned.pkl")
    print("✓ Tuned classifier načten")
except:
    try:
        classifier = joblib.load(f"{MODEL_PATH}/rf_classifier_all_sectors.pkl")
        print("✓ Base classifier načten")
    except:
        print("⚠️ Classifier nenalezen - spusťte nejprve Notebook 04")
        classifier = None

# Načtení hyperparametrů
try:
    with open(f"{MODEL_PATH}/optimal_hyperparameters.json", 'r') as f:
        optimal_params = json.load(f)
    print("✓ Hyperparametry načteny")
except:
    optimal_params = None
    print("⚠️ Hyperparametry nenalezeny")

---
## 3. Příprava Dat pro Evaluaci

In [ ]:
# Definice features
FEATURE_COLS = [
    'open', 'high', 'low', 'close', 'volume',
    'returns', 'volatility_12m', 'rsi_14',
    'macd', 'macd_signal', 'macd_hist',
    'sma_3', 'sma_6', 'sma_12',
    'ema_3', 'ema_6', 'ema_12',
    'volume_change', 'price_momentum'
]

FUNDAMENTAL_COLS = [
    'PE', 'PB', 'PS', 'EV_EBITDA',
    'ROE', 'ROA', 'Profit_Margin',
    'Debt_to_Equity', 'Current_Ratio',
    'Revenue_Growth_YoY', 'Earnings_Growth_YoY'
]

CLASS_NAMES = ['DOWN (< -3%)', 'HOLD (±3%)', 'UP (> +3%)']
THRESHOLD = 0.03

available_features = [f for f in FEATURE_COLS if f in df.columns]
available_fundamentals = [f for f in FUNDAMENTAL_COLS if f in df.columns]
all_features = available_features + available_fundamentals

In [ ]:
# Vytvoření target
df = df.sort_values(['ticker', 'date'])
df['future_return'] = df.groupby('ticker')['close'].shift(-1) / df['close'] - 1

conditions = [
    df['future_return'] < -THRESHOLD,
    df['future_return'] > THRESHOLD,
]
choices = [0, 2]
df['target'] = np.select(conditions, choices, default=1)

# Čištění
eval_df = df.dropna(subset=all_features + ['target']).copy()

print(f"📊 Evaluační data: {len(eval_df):,} záznamů")
print(f"\n📊 Distribuce tříd:")
for cls, name in enumerate(CLASS_NAMES):
    count = (eval_df['target'] == cls).sum()
    pct = count / len(eval_df) * 100
    print(f"   {name}: {count:,} ({pct:.1f}%)")

In [ ]:
# Chronologický split (80/20)
eval_df = eval_df.sort_values('date')
split_idx = int(len(eval_df) * 0.8)

train_df = eval_df.iloc[:split_idx]
test_df = eval_df.iloc[split_idx:]

X_train = train_df[all_features].values
y_train = train_df['target'].values.astype(int)

X_test = test_df[all_features].values
y_test = test_df['target'].values.astype(int)

# Standardizace
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"🔀 Train: {len(train_df):,} ({train_df['date'].min().strftime('%Y-%m')} - {train_df['date'].max().strftime('%Y-%m')})")
print(f"🔀 Test:  {len(test_df):,} ({test_df['date'].min().strftime('%Y-%m')} - {test_df['date'].max().strftime('%Y-%m')})")

---
## 4. Kompletní Evaluace Classifier

### 4.1 Základní Metriky

In [ ]:
# Predikce
if classifier is not None:
    y_pred = classifier.predict(X_test_scaled)
    y_pred_proba = classifier.predict_proba(X_test_scaled)
    
    print("📊 METRIKY KLASIFIKACE")
    print("="*60)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision_w = precision_score(y_test, y_pred, average='weighted')
    recall_w = recall_score(y_test, y_pred, average='weighted')
    f1_w = f1_score(y_test, y_pred, average='weighted')
    
    print(f"\n{'Metrika':<20} {'Hodnota':>15}")
    print("-"*40)
    print(f"{'Accuracy':<20} {accuracy:>15.4f}")
    print(f"{'Precision (weighted)':<20} {precision_w:>15.4f}")
    print(f"{'Recall (weighted)':<20} {recall_w:>15.4f}")
    print(f"{'F1-Score (weighted)':<20} {f1_w:>15.4f}")
else:
    print("⚠️ Classifier není k dispozici")

In [ ]:
# Classification Report
if classifier is not None:
    print("\n📊 CLASSIFICATION REPORT")
    print("="*60)
    print(classification_report(y_test, y_pred, target_names=CLASS_NAMES))

### 4.2 Confusion Matrix

In [ ]:
if classifier is not None:
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Absolutní hodnoty
    ax1 = axes[0]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
    ax1.set_xlabel('Predikované', fontweight='bold')
    ax1.set_ylabel('Skutečné', fontweight='bold')
    ax1.set_title('Confusion Matrix (absolutní)', fontweight='bold')
    
    # Normalizované
    ax2 = axes[1]
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues', ax=ax2,
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
    ax2.set_xlabel('Predikované', fontweight='bold')
    ax2.set_ylabel('Skutečné', fontweight='bold')
    ax2.set_title('Confusion Matrix (normalizované)', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f"{FIGURES_PATH}/confusion_matrix.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n💾 Graf uložen: {FIGURES_PATH}/confusion_matrix.png")

### 4.3 ROC Křivky (One-vs-Rest)

In [ ]:
if classifier is not None:
    # Binarizace pro ROC
    y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
    n_classes = 3
    
    # Výpočet ROC pro každou třídu
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Vizualizace
    fig, ax = plt.subplots(figsize=(10, 8))
    
    colors = ['#e41a1c', '#377eb8', '#4daf4a']
    for i, (color, name) in enumerate(zip(colors, CLASS_NAMES)):
        ax.plot(fpr[i], tpr[i], color=color, lw=2,
                label=f'{name} (AUC = {roc_auc[i]:.3f})')
    
    ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.500)')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontweight='bold')
    ax.set_ylabel('True Positive Rate', fontweight='bold')
    ax.set_title('ROC Křivky - Multiclass Classification', fontweight='bold', fontsize=14)
    ax.legend(loc='lower right', fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"{FIGURES_PATH}/roc_curves.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n💾 Graf uložen: {FIGURES_PATH}/roc_curves.png")

---
## 5. Porovnání Sektorů

In [ ]:
# Evaluace per sektor
if classifier is not None:
    sectors = test_df['sector'].unique()
    sector_metrics = []
    
    print("📊 METRIKY PO SEKTORECH")
    print("="*70)
    
    for sector in sectors:
        sector_mask = test_df['sector'] == sector
        sector_idx = test_df[sector_mask].index
        
        # Pozice v test_df
        pos = test_df.reset_index().index[test_df.reset_index()['sector'] == sector]
        
        y_sector = y_test[pos]
        pred_sector = y_pred[pos]
        
        acc = accuracy_score(y_sector, pred_sector)
        f1 = f1_score(y_sector, pred_sector, average='weighted')
        
        sector_metrics.append({
            'sector': sector,
            'samples': len(y_sector),
            'accuracy': acc,
            'f1_score': f1
        })
        
        print(f"\n🏢 {sector}:")
        print(f"   Samples: {len(y_sector):,}")
        print(f"   Accuracy: {acc:.4f}")
        print(f"   F1-Score: {f1:.4f}")
    
    sector_df = pd.DataFrame(sector_metrics)

In [ ]:
# Vizualizace porovnání sektorů
if classifier is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy
    ax1 = axes[0]
    bars1 = ax1.bar(sector_df['sector'], sector_df['accuracy'], color=['#3498db', '#2ecc71', '#e74c3c'])
    ax1.axhline(y=accuracy, color='k', linestyle='--', label=f'Celková: {accuracy:.3f}')
    ax1.set_ylabel('Accuracy', fontweight='bold')
    ax1.set_title('Accuracy po Sektorech', fontweight='bold')
    ax1.legend()
    for bar, val in zip(bars1, sector_df['accuracy']):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                 f'{val:.3f}', ha='center', fontweight='bold')
    
    # F1-Score
    ax2 = axes[1]
    bars2 = ax2.bar(sector_df['sector'], sector_df['f1_score'], color=['#3498db', '#2ecc71', '#e74c3c'])
    ax2.axhline(y=f1_w, color='k', linestyle='--', label=f'Celková: {f1_w:.3f}')
    ax2.set_ylabel('F1-Score', fontweight='bold')
    ax2.set_title('F1-Score po Sektorech', fontweight='bold')
    ax2.legend()
    for bar, val in zip(bars2, sector_df['f1_score']):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                 f'{val:.3f}', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f"{FIGURES_PATH}/sector_comparison.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n💾 Graf uložen: {FIGURES_PATH}/sector_comparison.png")

---
## 6. Feature Importance Analýza

In [ ]:
if classifier is not None and hasattr(classifier, 'feature_importances_'):
    # Feature importance
    importances = classifier.feature_importances_
    feature_names = all_features
    
    # Seřazení
    indices = np.argsort(importances)[::-1]
    
    # Top 15 features
    top_n = 15
    top_indices = indices[:top_n]
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    colors = ['#2ecc71' if 'PE' in feature_names[i] or 'ROE' in feature_names[i] 
              or 'Revenue' in feature_names[i] or 'Debt' in feature_names[i]
              else '#3498db' for i in top_indices]
    
    bars = ax.barh(range(top_n), importances[top_indices][::-1], color=colors[::-1])
    ax.set_yticks(range(top_n))
    ax.set_yticklabels([feature_names[i] for i in top_indices[::-1]])
    ax.set_xlabel('Importance', fontweight='bold')
    ax.set_title('Top 15 Feature Importance', fontweight='bold', fontsize=14)
    
    # Legenda
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='#3498db', label='Technical'),
                       Patch(facecolor='#2ecc71', label='Fundamental')]
    ax.legend(handles=legend_elements, loc='lower right')
    
    plt.tight_layout()
    plt.savefig(f"{FIGURES_PATH}/feature_importance.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n💾 Graf uložen: {FIGURES_PATH}/feature_importance.png")

---
## 7. Backtesting Obchodní Strategie

### 7.1 Definice Strategie

**Pravidla:**
- Predikce **UP (2)** → BUY (long pozice)
- Predikce **DOWN (0)** → SHORT (short pozice)
- Predikce **HOLD (1)** → Žádná akce

**Zjednodušení:**
- Ignorujeme transakční náklady (již zahrnuty v 3% thresholdu)
- Fixní velikost pozice

In [ ]:
if classifier is not None:
    # Backtesting
    backtest_df = test_df.copy().reset_index(drop=True)
    backtest_df['prediction'] = y_pred
    backtest_df['actual_return'] = backtest_df['future_return']
    
    # Strategy returns
    # UP (2) -> go long -> return = actual_return
    # DOWN (0) -> go short -> return = -actual_return
    # HOLD (1) -> no position -> return = 0
    
    def calc_strategy_return(row):
        if pd.isna(row['actual_return']):
            return 0
        if row['prediction'] == 2:  # UP -> long
            return row['actual_return']
        elif row['prediction'] == 0:  # DOWN -> short
            return -row['actual_return']
        else:  # HOLD
            return 0
    
    backtest_df['strategy_return'] = backtest_df.apply(calc_strategy_return, axis=1)
    
    # Kumulativní výnosy
    backtest_df['cumulative_strategy'] = (1 + backtest_df['strategy_return']).cumprod()
    backtest_df['cumulative_buyhold'] = (1 + backtest_df['actual_return'].fillna(0)).cumprod()
    
    print("📊 BACKTESTING VÝSLEDKY")
    print("="*60)

In [ ]:
if classifier is not None:
    # Statistiky strategie
    total_trades = (backtest_df['prediction'] != 1).sum()
    long_trades = (backtest_df['prediction'] == 2).sum()
    short_trades = (backtest_df['prediction'] == 0).sum()
    
    strategy_return = backtest_df['cumulative_strategy'].iloc[-1] - 1
    buyhold_return = backtest_df['cumulative_buyhold'].iloc[-1] - 1
    
    # Winning trades
    winning_trades = ((backtest_df['prediction'] != 1) & (backtest_df['strategy_return'] > 0)).sum()
    win_rate = winning_trades / total_trades * 100 if total_trades > 0 else 0
    
    # Sharpe Ratio (simplified)
    strategy_returns = backtest_df['strategy_return']
    sharpe = strategy_returns.mean() / strategy_returns.std() * np.sqrt(12) if strategy_returns.std() > 0 else 0
    
    print(f"\n📊 Statistiky:")
    print(f"   Celkem obchodů: {total_trades:,}")
    print(f"   Long pozice: {long_trades:,}")
    print(f"   Short pozice: {short_trades:,}")
    print(f"   Win Rate: {win_rate:.1f}%")
    
    print(f"\n📈 Výnosy:")
    print(f"   Strategie: {strategy_return*100:.2f}%")
    print(f"   Buy & Hold: {buyhold_return*100:.2f}%")
    print(f"   Outperformance: {(strategy_return - buyhold_return)*100:.2f}%")
    
    print(f"\n📊 Rizikové metriky:")
    print(f"   Sharpe Ratio: {sharpe:.3f}")

In [ ]:
if classifier is not None:
    # Vizualizace equity curve
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Equity Curve
    ax1 = axes[0]
    ax1.plot(backtest_df.index, backtest_df['cumulative_strategy'], 
             label='ML Strategie', linewidth=2, color='#2ecc71')
    ax1.plot(backtest_df.index, backtest_df['cumulative_buyhold'], 
             label='Buy & Hold', linewidth=2, color='#3498db', alpha=0.7)
    ax1.axhline(y=1, color='k', linestyle='--', alpha=0.3)
    ax1.set_ylabel('Portfolio Value', fontweight='bold')
    ax1.set_title('Equity Curve: ML Strategie vs Buy & Hold', fontweight='bold', fontsize=14)
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Drawdown
    ax2 = axes[1]
    rolling_max = backtest_df['cumulative_strategy'].expanding().max()
    drawdown = (backtest_df['cumulative_strategy'] - rolling_max) / rolling_max * 100
    ax2.fill_between(backtest_df.index, drawdown, 0, alpha=0.5, color='#e74c3c')
    ax2.set_ylabel('Drawdown (%)', fontweight='bold')
    ax2.set_xlabel('Index', fontweight='bold')
    ax2.set_title('Drawdown', fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"{FIGURES_PATH}/backtest_equity.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n💾 Graf uložen: {FIGURES_PATH}/backtest_equity.png")

---
## 8. Souhrn Celého Projektu

In [ ]:
# Finální souhrn
print("="*70)
print("📊 FINÁLNÍ SOUHRN PROJEKTU")
print("Klasifikace Cenových Pohybů Akcií pomocí Machine Learning")
print("="*70)

print(f"\n📅 Data:")
print(f"   Období: 2015-2025 (10 let)")
print(f"   Tickerů: ~150 (S&P 500)")
print(f"   Sektory: Technology, Consumer, Industrials")

print(f"\n🧠 Modely:")
print(f"   1. RF Regressor - imputace fundamentálních dat (2015-2024)")
print(f"   2. RF Classifier - ternární klasifikace (DOWN/HOLD/UP)")

print(f"\n📊 Výsledky Classifier:")
if classifier is not None:
    print(f"   Accuracy: {accuracy:.4f}")
    print(f"   F1-Score: {f1_w:.4f}")
    print(f"   Win Rate: {win_rate:.1f}%")

print(f"\n💡 Klíčová zjištění:")
print(f"   • Random Forest poskytuje robustní klasifikaci")
print(f"   • 3% threshold pokrývá transakční náklady")
print(f"   • Fundamentální data zlepšují predikci")
print(f"   • TimeSeriesSplit je kritický pro validní evaluaci")

In [ ]:
# Export výsledků
results_summary = {
    'project': 'Stock Price Movement Classification',
    'author': 'Bc. Jan Dub',
    'date': datetime.now().isoformat(),
    'data': {
        'period': '2015-2025',
        'tickers': 150,
        'sectors': ['Technology', 'Consumer', 'Industrials'],
        'total_records': len(df)
    },
    'classification': {
        'threshold': THRESHOLD,
        'classes': CLASS_NAMES,
        'accuracy': float(accuracy) if classifier else None,
        'f1_weighted': float(f1_w) if classifier else None,
        'precision_weighted': float(precision_w) if classifier else None,
        'recall_weighted': float(recall_w) if classifier else None
    },
    'backtest': {
        'total_trades': int(total_trades) if classifier else None,
        'win_rate': float(win_rate) if classifier else None,
        'strategy_return': float(strategy_return) if classifier else None,
        'buyhold_return': float(buyhold_return) if classifier else None,
        'sharpe_ratio': float(sharpe) if classifier else None
    }
}

results_path = f"{DATA_PATH}/final_results.json"
with open(results_path, 'w') as f:
    json.dump(results_summary, f, indent=2)

print(f"\n💾 Výsledky exportovány: {results_path}")

---
## 9. Seznam Generovaných Grafů

Pro použití v diplomové práci:

| Soubor | Popis |
|--------|-------|
| `confusion_matrix.png` | Confusion matrix (absolutní + normalizovaná) |
| `roc_curves.png` | ROC křivky pro všechny třídy |
| `sector_comparison.png` | Porovnání sektorů (Accuracy, F1) |
| `feature_importance.png` | Top 15 nejdůležitějších features |
| `backtest_equity.png` | Equity curve + Drawdown |

---

## ✅ PROJEKT DOKONČEN

Všechny notebooky jsou připraveny pro spuštění na Google Colab.

In [ ]:
print("\n" + "="*70)
print("✅ NOTEBOOK 06 - FINAL EVALUATION DOKONČEN")
print("="*70)
print(f"\n📁 Vytvořené soubory:")
print(f"   • {FIGURES_PATH}/confusion_matrix.png")
print(f"   • {FIGURES_PATH}/roc_curves.png")
print(f"   • {FIGURES_PATH}/sector_comparison.png")
print(f"   • {FIGURES_PATH}/feature_importance.png")
print(f"   • {FIGURES_PATH}/backtest_equity.png")
print(f"   • {DATA_PATH}/final_results.json")
print(f"\n🎓 Grafy jsou připraveny pro diplomovou práci!")